In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from datetime import timedelta

pd.options.display.max_columns = None

#data dictionaries
import state_abbreviations
import statepop_2019est

X:\Users\AC\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

url3 = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url3,error_bad_lines = False)
df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})
df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']] = df_testing[['totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative']].fillna(0)

In [3]:
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    """Checks whether the daily increments column adds up to the cumulative column for testing, positive, and negative results.
    
    If the columns are not equal, ID's the earliest date which the numbers are misaligned and fills the incremental results with the cumulative results.
    After review, the testing df is only missing the first daily increments.
    Still might want to add a check in case filling the earliest date is not enough to equate the two columns."""
    for state in states:
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])] 

In [4]:
fill_func(df_testing['Province_State'].unique())

#Update state names
df_testing = df_testing.replace({'Province_State':state_abbreviations.state_abbrev})

#add state populations
df_testing['Population'] = df_testing['Province_State'].map(statepop_2019est.statepop_2019est)

df_testing['Date'] = pd.to_datetime(df_testing['Date'].astype(dtype = 'str'))

In [5]:
ca_testing = df_testing[['Date','Province_State','positive','negative','hospitalizedCurrently','inIcuCurrently', 'death', 'positiveIncrease','negativeIncrease','totalTestResults','totalTestResultsIncrease','deathIncrease']].loc[df_testing['Province_State'] =='California'].copy().reset_index(drop=True)
ca_testing['Province_State'] = 'California'
ca_testing = ca_testing.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_testing = df_testing[['Province_State','Date','Population','positiveIncrease', 'totalTestResultsIncrease','totalTestResults','positive','negative']].copy()

In [6]:
#Feature creation
us_testing['Tests per 1000 Capita'] = us_testing['totalTestResults']/us_testing['Population']*1000
us_testing['Total Positivity Rate'] = us_testing['positive']/us_testing['totalTestResults']*100

ranks = []
for date in us_testing['Date'].unique():
    for ranking in us_testing.copy().loc[us_testing['Date']==date,'Tests per 1000 Capita'].rank(ascending = False, method='min'):
        ranks.append(ranking)
rankings = pd.DataFrame({'Tests per Capita Daily Ranking': ranks})
us_testing = pd.concat([us_testing, rankings], axis=1)

In [7]:
check = 0
for state in df_testing['Province_State'].unique():
    for col in range(0,len(to_fix)):
        cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
        actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
        comparison = cumsum.eq(actualsum)
        if comparison[comparison == False].count() > 0:
            print(state, to_fix[col], actual[col])
            check += 1
if check == 0:
    print('All States checked')

All States checked


In [8]:
ca_testing.tail()

,Date,Province_State,positive,negative,hospitalizedCurrently,inIcuCurrently,death,positiveIncrease,negativeIncrease,totalTestResults,totalTestResultsIncrease,deathIncrease
227,2020-10-17,California,864455.0,15892256.0,2995.0,743.0,16899.0,2979.0,131776.0,16756711.0,134755.0,69
228,2020-10-18,California,867317.0,16024745.0,3020.0,727.0,16943.0,2862.0,132489.0,16892062.0,135351.0,44
229,2020-10-19,California,870791.0,16171617.0,3002.0,758.0,16970.0,3474.0,146872.0,17042408.0,150346.0,27
230,2020-10-20,California,874077.0,16314993.0,3104.0,781.0,16992.0,3286.0,143376.0,17189070.0,146662.0,22
231,2020-10-21,California,877784.0,16415355.0,3077.0,763.0,17027.0,3707.0,100362.0,17293139.0,104069.0,35


In [9]:
ca_testing.to_csv(r'X:\AC\Documents\Datasets\CA_daily_pivot.csv', index=False)
us_testing.to_csv(r'X:\AC\Documents\Datasets\US_testing_daily.csv', index=False)

# Data Exploration

In [10]:
most_recent = df_testing.loc[(df_testing['Province_State']=='California')]['Date'].max()
last14 = [most_recent - timedelta(days=x) for x in range(14)]
most_recent = df_testing.loc[(df_testing['Province_State']=='California') & (df_testing['Date']==most_recent)]
last14 = df_testing.loc[(df_testing['Province_State']=='California') & (df_testing['Date'].isin(last14))]

current_hosp = most_recent['hospitalizedCurrently'].iloc[0].astype(int)
perc_pos = (most_recent['positive']/most_recent['totalTestResults']).iloc[0] * 100
total_tests = most_recent['totalTestResults'].iloc[0]
population = statepop_2019est.statepop_2019est['California']
testsper1000 = total_tests/population*1000

last14tests = last14['totalTestResultsIncrease'].sum()
last14perc_pos = last14['positiveIncrease'].sum()/last14tests * 100

print('California Stats as of: ', pd.datetime.strftime(most_recent['Date'].max(), '%m/%d/%y'))
print('Total population in CA: %d' %population)
print('\n')
print('Total tests conducted: %d' %total_tests)
print('Total tests conducted per 1000 people in CA: %.2f' %testsper1000)
print('Total cases per 1000 people in CA: %.2f' %(most_recent['positive']/population*1000))
print('Percentage of total tests confirmed positive: %.2f' %perc_pos,'%')
print('Percentage of total tests confirmed negative: %.2f' %(100 - perc_pos),'%')
print(f'Currently hospitalized: {current_hosp}')
print('Days recorded: {}'.format(len(df_testing['Date'].unique())))
print('\n')
print('Last 14 days:')
print('Total tests conducted in last 14 days: %d' %last14tests)
print('Percentage of tests in last 14 days confirmed positive: %.2f' %last14perc_pos,'%')
print('Tests conducted in last 14 days as percentage of totals tests conducted: %d' %(last14tests/total_tests*100), '%')
print('Positives in last 14 days as percentage of total positives: %.2f' %(last14['positiveIncrease'].sum()/most_recent['positive']*100),'%')

California Stats as of:  10/21/20
Total population in CA: 39512223


Total tests conducted: 17293139
Total tests conducted per 1000 people in CA: 437.67
Total cases per 1000 people in CA: 22.22
Percentage of total tests confirmed positive: 5.08 %
Percentage of total tests confirmed negative: 94.92 %
Currently hospitalized: 3077
Days recorded: 274


Last 14 days:
Total tests conducted in last 14 days: 1736433
Percentage of tests in last 14 days confirmed positive: 2.68 %
Tests conducted in last 14 days as percentage of totals tests conducted: 10 %
Positives in last 14 days as percentage of total positives: 5.30 %


X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from ipykernel import kernelapp as app


In [11]:
CA_greatest_daily = us_testing.loc[us_testing['Province_State']=='California'].sort_values('positiveIncrease', ascending = False).iloc[0]

print('Highest Daily Increase for California on: ', pd.datetime.strftime(CA_greatest_daily['Date'], '%m/%d/%y'))
print('Positive cases: %d' %(CA_greatest_daily['positiveIncrease']))
print('Positivity rate: %.2f' %(CA_greatest_daily['positiveIncrease']/CA_greatest_daily['totalTestResultsIncrease']*100),'%')

Highest Daily Increase for California on:  07/22/20
Positive cases: 12807
Positivity rate: 10.05 %


X:\Users\AC\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
us_testing.loc[us_testing['Date']==us_testing['Date'].max()].sort_values('Population', ascending = False)

,Province_State,Date,Population,positiveIncrease,totalTestResultsIncrease,totalTestResults,positive,negative,Tests per 1000 Capita,Total Positivity Rate,Tests per Capita Daily Ranking
5,California,2020-10-21,39512223,3707.0,104069.0,17293139.0,877784.0,16415355.0,437.665555,5.075909,17.0
47,Texas,2020-10-21,28995881,5252.0,30548.0,7474874.0,838809.0,6636065.0,257.790891,11.221714,47.0
10,Florida,2020-10-21,21477737,0.0,0.0,5767947.0,760389.0,5007558.0,268.554690,13.183009,43.0
37,New York,2020-10-21,19453561,2026.0,124789.0,13197504.0,488506.0,12708998.0,678.410703,3.701503,6.0
41,Pennsylvania,2020-10-21,12801989,1425.0,13322.0,2353833.0,186297.0,2167536.0,183.864632,7.914623,54.0
16,Illinois,2020-10-21,12671821,4342.0,66791.0,6953692.0,358804.0,6594888.0,548.752385,5.159906,10.0
38,Ohio,2020-10-21,11689100,2366.0,32099.0,4011513.0,188005.0,3823508.0,343.184077,4.686636,29.0
11,Georgia,2020-10-21,10617423,0.0,0.0,3327836.0,342438.0,2985398.0,313.431611,10.290110,34.0
30,North Carolina,2020-10-21,10488084,1842.0,20250.0,3692043.0,250592.0,3441451.0,352.022638,6.787353,28.0
24,Michigan,2020-10-21,9986857,1878.0,35040.0,4420831.0,167872.0,4252959.0,442.664894,3.797295,16.0


# Forecasting

In [28]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA

from datetime import datetime

In [34]:
ca_cases = us_testing.loc[us_testing['Province_State']=='California', ['Date','positive']]
type(ca_cases['Date'].iloc[0])
# ca_log = np.log(ca_cases)
# plt.plot(ca_cases)

pandas._libs.tslibs.timestamps.Timestamp

# Appendix

In [ ]:
us_conf_daily = df_usconf.copy()
us_dead_daily = df_usdead.copy()

us_conf_daily.iloc[:,11:] = us_conf_daily.iloc[:,11:].diff(axis=1).fillna(0).astype('int')
us_dead_daily.iloc[:,12:] = us_dead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

us_conf_daily = us_conf_daily.groupby(['Province_State'])[us_conf_daily.columns.to_list()[11:]].agg('sum').reset_index()
us_dead_daily = us_dead_daily.groupby(['Province_State'])[us_dead_daily.columns.to_list()[11:]].agg('sum').reset_index()

us_conf_daily = us_conf_daily.melt(id_vars = us_conf_daily.columns[:1], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['Province_State','Date']).reset_index(drop=True)
us_dead_daily = us_dead_daily.melt(id_vars = us_dead_daily.columns[:2], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_daily = us_conf_daily.merge(us_dead_daily[['Province_State','Population','Date']], on = ['Province_State','Date'], how = 'left', suffixes = (False, False))
us_daily = us_daily[['Province_State','Population','Date']]
us_daily['Date'] = pd.to_datetime(us_daily['Date'])
us_daily = us_daily.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

In [ ]:
#there will be discrepancies between the daily confirmed cases from JHU and from CDHP.
#Should probably use all CDHP metrics for CA analysis and only population from JHU
ca_daily = us_daily.loc[us_daily['Province_State']=='California'].copy().reset_index(drop=True)
ca_daily = ca_daily.merge(ca_testing[['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']], on = ['Province_State', 'Date'], how = 'left', suffixes = (False, False))

us_daily = df_testing[['Province_State','Date','positiveIncrease','negativeIncrease', 'totalTestResultsIncrease','totalTestResults','positive','negative']]

In [ ]:
df_pop = pd.read_csv(r'X:\AC\Documents\Datasets\US Census and OMB Data\2019 State Population Estimates.csv', delimiter = ',', encoding = "ISO-8859-1")

In [ ]:
pop_check = df_pop.merge(us_testing[['Province_State','Population']], how='inner', left_on='State Name', right_on='Province_State')

In [ ]:
pop_check.drop_duplicates(inplace=True)
pop_check['delta'] = pop_check['2019 Total Population Estimate'] - pop_check['Population']
pop_check.loc[pop_check['delta']!=0]

In [ ]:
#apply date format and convert to string
.apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))

In [ ]:
columns = test_df.columns
test_df.iloc[7694,columns.get_loc('totalTestResultsIncrease')] = 1
a = np.cumsum(test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResultsIncrease'].values,dtype=int)
b = test_df.loc[test_df['Province_State']=='TX'].sort_values(by='Date')['totalTestResults'].values
np.array_equal(a,b)

In [ ]:
#filling missing starting values. May want to do this as function for all states
df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positiveIncrease','negativeIncrease','totalTestResultsIncrease']] = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positive','negative','totalTestResults']].values.tolist()

In [ ]:
#Update function with secondary check
to_fix = ['totalTestResultsIncrease','positiveIncrease','negativeIncrease']
actual = ['totalTestResults','positive','negative']
columns = df_testing.columns
def fill_func(states):
    for state in states:
        print(state)
        for col in range(0,len(to_fix)):
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            print(cumsum.iloc[0], actualsum.iloc[0], comparison.iloc[0])
            if comparison[comparison == False].count() > 0:
                Earliest_index = comparison[comparison == False].index[0]
                df_testing.iloc[Earliest_index,columns.get_loc(to_fix[col])] = df_testing.iloc[Earliest_index,columns.get_loc(actual[col])]
                print('Fixed in first: ', state,to_fix[col],actual[col])
            else:
                print('Arrays are equal')
            cumsum = df_testing.loc[df_testing['Province_State']==state].sort_values(by='Date')[to_fix[col]].cumsum()
            actualsum = df_testing.loc[df_testing['Province_State']==state,['Date',actual[col]]].sort_values(by='Date')[actual[col]]
            comparison = cumsum.eq(actualsum)
            if comparison[comparison == False].count() == 0:
                print('Confirmed arrays are fixed')
            else:
                print('Failed check: ', state,to_fix[col],actual[col] ,'\n')

In [ ]:
#['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']
#hospitalized currently missing values from american samoa, Hawaii, Kansas, northern marinas, virginia
#inincu currently missing values from 28 states/provinces
df_testing.loc[df_testing['Date']=='2020-07-21',['Province_State', 'Date','totalTestResultsIncrease','positiveIncrease','negativeIncrease','totalTestResults','positive','negative','hospitalizedCurrently','inIcuCurrently']]

In [ ]:
cumsum = df_testing.loc[df_testing['Province_State']=='NV'].sort_values(by='Date')['totalTestResultsIncrease'].cumsum()
actualsum = df_testing.loc[df_testing['Province_State']=='NV',['Date','totalTestResults']].sort_values(by='Date')['totalTestResults']
check = cumsum.eq(actualsum)
check[check==False].sum()

In [ ]:
#Feature creation, rankings
#Difference between daily rankings
dates = stategrp['Date'].unique().tolist()
ustemp = pd.DataFrame()
for state in stategrp['Province_State'].unique():
    statetemp = stategrp.loc[stategrp['Province_State']==state].copy().reset_index(drop=True)
    yest = pd.Series(statetemp.loc[statetemp['Date'].isin(dates[1:]),'Total Cases Daily Ranking'].reset_index(drop=True)).rename('Total Cases Ranking Daily Change')
    statetemp = pd.concat([statetemp,yest], axis=1)
    ustemp = pd.concat([ustemp, statetemp])

In [4]:
state_abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [5]:
statepop_2019est = {
        'Alaska': 731545,
        'Alabama': 4903185,
        'Arkansas': 3017804,
        'American Samoa': 55312,
        'Arizona': 7278717,
        'California': 39512223,
        'Colorado': 5758736,
        'Connecticut': 3565287,
        'District of Columbia': 705749,
        'Delaware': 973764,
        'Florida': 21477737,
        'Georgia': 10617423,
        'Guam': 167294,
        'Hawaii': 1415872,
        'Iowa': 3155070,
        'Idaho': 1787065,
        'Illinois': 12671821,
        'Indiana': 6732219,
        'Kansas': 2913314,
        'Kentucky': 4467673,
        'Louisiana': 4648794,
        'Massachusetts': 6892503,
        'Maryland': 6045680,
        'Maine': 1344212,
        'Michigan': 9986857,
        'Minnesota': 5639632,
        'Missouri': 6137428,
        'Northern Mariana Islands': 57216,
        'Mississippi': 2976149,
        'Montana': 1068778,
        'North Carolina': 10488084,
        'North Dakota': 762062,
        'Nebraska': 1934408,
        'New Hampshire': 1359711,
        'New Jersey': 8882190,
        'New Mexico': 2096829,
        'Nevada': 3080156,
        'New York': 19453561,
        'Ohio': 11689100,
        'Oklahoma': 3956971,
        'Oregon': 4217737,
        'Pennsylvania': 12801989,
        'Puerto Rico': 2933408,
        'Rhode Island': 1059361,
        'South Carolina': 5148714,
        'South Dakota': 884659,
        'Tennessee': 6829174,
        'Texas': 28995881,
        'Utah': 3205958,
        'Virginia': 8535519,
        'Virgin Islands': 30030,
        'Vermont': 623989,
        'Washington': 7614893,
        'Wisconsin': 5822434,
        'West Virginia': 1792147,
        'Wyoming': 578759}